# RSA Communication Scheme


In this notebook, I am gonna show how to:

- Setup a RSA communication scheme and use it to securely exchange messages over an insecure channel
- Use RSA for Digital Signature (Digital Identity, Digital Intent)

This implementation is based on the following textbook: 

```Understanding Cryptography: Pr. Christof Paar. & Pr. Jan Pelzl. Springer 2010, 1st edition, Chapters 6, 7 and 11```

## RSA Key Generation


![RSA key generation](001.png)


## Encryption and Decryption functions



![RSA encryption](005.png "RSA encryption")
![RSA decryption](006.png "RSA decryption")


## Numerical Example



Before diving into the coding part, let's use the 2 examples found in the course to illustrate the RSA communication scheme and the underlying challenges its implementation poses.

### Example 1: Atomic values



In the textbook, the following values are proposed as an example:

- _Step 1:_ $p=3$ and $q=11$
- _Step 2:_ $n=p.q=3.11=33$
- _Step 3:_ $\phi(n)=(p-1)(q-1)=2.10=20$
- _Step 4:_ $e=3 \in \{1, 2, ..., 19\}$ and confirm that $gcd(3, 20) = 1$
- _Step 5:_ $d=7$ satisfies the relation $d.e=mod\ \phi(n) \leftrightarrow 3 \cdot 7=21=mod(20)$

Using the public key $k_{pub}=(n,e)=(20,3)$ Alice can send to Bob a message $x=17 \in Z_{20}$ by encrypting it: 
$$y=x^e \ mod(n)=17^3 \ mod(20)=4913 \ mod(20)=13$$

Bob can decrypt the message using his private key $d=7$:
$$x=y^d \ mod(n)=13^7 \ mod(20)=62,748,517 \ mod(20)=17$$

### Example 2: Real World values and the need for fast computation algorithms



The second illustration from the book uses real world values with a key size of 1024 bits (p & q are 512 bits each):

In [6]:
# Step 1: select p & q
p=0xE0DFD2C2A288ACEBC705EFAB30E4447541A8C5A47A37185C5A9CB98389CE4DE19199AA3069B404FD98C801568CB9170EB712BF10B4955CE9C9DC8CE6855C6123;
q=0xEBE0FCF21866FD9A9F0D72F7994875A8D92E67AEE4B515136B2A778A8048B149828AEA30BD0BA34B977982A3D42168F594CA99F3981DDABFAB2369F229640115;

# Step 2: compute n
n=p*q;

# Step 3: compute phi(n)
phi=(p-1)*(q-1);

# Step 4: select e
e=0x40B028E1E4CCF07537643101FF72444A0BE1D7682F1EDB553E3AB4F6DD8293CA1945DB12D796AE9244D60565C2EB692A89B8881D58D278562ED60066DD8211E67315CF89857167206120405B08B54D10D4EC4ED4253C75FA74098FE3F7FB751FF5121353C554391E114C85B56A9725E9BD5685D6C9C7EED8EE442366353DC39;

# check if e<phi(n) (we will define below the Extended Euclidian Algoritm to find the gcd of 2 numbers)
print("+ is e < phi(n)=",e<phi);

# Step 5: compute d (same remark; we will use EEA algorithm later to get the inverse of e)
d=0xC21A93EE751A8D4FBFD77285D79D6768C58EBF283743D2889A395F266C78F4A28E86F545960C2CE01EB8AD5246905163B28D0B8BAABB959CC03F4EC499186168AE9ED6D88058898907E61C7CCCC584D65D801CFE32DFC983707F87F5AA6AE4B9E77B9CE630E2C0DF05841B5E4984D059A35D7270D500514891F7B77B804BED81;

# Alice sends an encrypted message to Bob
x=0xFEA1490B5F3D8E1;
y=(x**e)%n;
print("Alice send a message to Bob.");
print("x=",hex(x));
print("y=",hex(y));

# Bob decodes it
x_=(y**d)%n;
print("Bob decrypts the message=",hex(x_));

+ is e < phi(n)= True


KeyboardInterrupt: 

Without any surprise, the naïve approach consisting of encrypting and decrypting by simply computing the exponent of such large numbers using the $**$ operator will take ages ($2^{512}$ exponentiation operations when there are $2^{300}$ atoms in the visible universe). This illustrate the need for fast computing methods.

Since computing an exponentiation is central to all the further calculations, let's see how we can speed up the process.

# Implementation

## Step 0: Practical Modular Exponentiation

As mentioned, before considering implementing the RSA communication scheme, we should make sure to have a practical way of computing modular exponentiations. By practical, we mean an algorithm that has a polynomial complexity as opposed to an exponential complexity as we currently have with the "**" operator.

![Square-and-Multiply Modular Exponentiation](004.png "Square-and-Multiply Modular Exponentiation")

In [46]:
def square_and_multiply(x:int, H:int, n:int)->int:
    r=x 
    l=len(bin(H))-2;
    for i in range(1,l):
        r=(r**2)%n;
        hi=bin(H)[i+2]; # ignore the prefix 0b
        if hi == "1":
            r=(r*x)%n
    return r

### Testing the Square-and-Multiply function

Now that we have a practical/more efficient way of computing modular exponent, let's test it on the real life book example:



In [98]:
# Step 1: select p & q
p=0xE0DFD2C2A288ACEBC705EFAB30E4447541A8C5A47A37185C5A9CB98389CE4DE19199AA3069B404FD98C801568CB9170EB712BF10B4955CE9C9DC8CE6855C6123;
q=0xEBE0FCF21866FD9A9F0D72F7994875A8D92E67AEE4B515136B2A778A8048B149828AEA30BD0BA34B977982A3D42168F594CA99F3981DDABFAB2369F229640115;
print("-Bob selects p=", hex(p));
print("-Bob selects q=", hex(q));

# Step 2: compute n
n=p*q;
print("-Bob computes n=p.q=", hex(n));

# Step 3: compute phi(n)
phi=(p-1)*(q-1);
print("-Bob computes phi(n)=(p-1)*(q-1)=", hex(phi));

# Step 4: select e
e=0x40B028E1E4CCF07537643101FF72444A0BE1D7682F1EDB553E3AB4F6DD8293CA1945DB12D796AE9244D60565C2EB692A89B8881D58D278562ED60066DD8211E67315CF89857167206120405B08B54D10D4EC4ED4253C75FA74098FE3F7FB751FF5121353C554391E114C85B56A9725E9BD5685D6C9C7EED8EE442366353DC39;
print("-Bob selects e=", hex(e));

# Step 5: compute d (same remark; we will use EEA algorithm later to get the inverse of e)
d=0xC21A93EE751A8D4FBFD77285D79D6768C58EBF283743D2889A395F266C78F4A28E86F545960C2CE01EB8AD5246905163B28D0B8BAABB959CC03F4EC499186168AE9ED6D88058898907E61C7CCCC584D65D801CFE32DFC983707F87F5AA6AE4B9E77B9CE630E2C0DF05841B5E4984D059A35D7270D500514891F7B77B804BED81;
print("-Bob computes d=", hex(d));

# B sends the public key to alice
print("- Bob sends the public key (n,e) to Alice");

# Alice sends an encrypted message to Bob
x=0xFEA1490B5F3D8E1;
y=square_and_multiply(x, e, n);
print("-Alice sends a message to Bob.");
print("clear-text x=",hex(x));
print("cipher-text y=",hex(y));

# Bob decodes the message
x_=square_and_multiply(y, d, n);
print("-Bob decrypts the cipher-text and obtains the clear-text=",hex(x_));

-Bob selects p= 0xe0dfd2c2a288acebc705efab30e4447541a8c5a47a37185c5a9cb98389ce4de19199aa3069b404fd98c801568cb9170eb712bf10b4955ce9c9dc8ce6855c6123
-Bob selects q= 0xebe0fcf21866fd9a9f0d72f7994875a8d92e67aee4b515136b2a778a8048b149828aea30bd0ba34b977982a3d42168f594ca99f3981ddabfab2369f229640115
-Bob computes n=p.q= 0xcf33188211fdf6052bdbb1a37235e0abb5978a45c71fd381a91ad12fc76da0544c47568ac83d855d47ca8d8a779579ab72e635d0b0aaac22d28341e998e90f82122a2c06090f43a37e0203c2b72e401fd06890ec8ead4f07e686e906f01b2468ae7b30cbd670255c1fede1a2762cf4392c0759499cc0abecff008728d9a11adf
-Bob computes phi(n)=(p-1)*(q-1)= 0xcf33188211fdf6052bdbb1a37235e0abb5978a45c71fd381a91ad12fc76da0544c47568ac83d855d47ca8d8a779579ab72e635d0b0aaac22d28341e998e90f8045695c514e1f991d17eea11fed018601b59163992fc1219820bfb7f8e604253d9a569c6aafb07d12efac5da815527434e02a0045500d74438a0090502ae0b8a8
-Bob selects e= 0x40b028e1e4ccf07537643101ff72444a0be1d7682f1edb553e3ab4f6dd8293ca1945db12d796ae9244d60565c2eb692a89b8881d58d278562ed

## Step 1: Choosing large prime numbers


In order to generate large prime number, we are going to use the *Fermat Primality Test*:

![Fermat Primality Test](002.png "Fermat Primality Test")

This is based on *Fermat's Little Theorem*:

![Fermat's Little Theorem](003.png "Fermat's Little Theorem")

In [55]:
import random;

def fermat_primality_test(p_:int,s:int=2)->bool:
    for i in range(1,s+1):
        a=random.randint(2, p_-2);
        if square_and_multiply(a,p_-1,p_) !=1:
            return False
    return True

### Testing Fermat Primality Test algorithm

Let's test the algorithm with the real world values from the text book using a relatively high security factor:

In [72]:
p=0xE0DFD2C2A288ACEBC705EFAB30E4447541A8C5A47A37185C5A9CB98389CE4DE19199AA3069B404FD98C801568CB9170EB712BF10B4955CE9C9DC8CE6855C6123;
q=0xEBE0FCF21866FD9A9F0D72F7994875A8D92E67AEE4B515136B2A778A8048B149828AEA30BD0BA34B977982A3D42168F594CA99F3981DDABFAB2369F229640115;

print("Is p prime?",fermat_primality_test(p,1000));
print("Is q prime?",fermat_primality_test(q,1000));

2048
Is p prime? True
Is q prime? True


We can now use the algorithm to generate 2 large primes p & q. The procedure works as follow:

1. Generate a random 512 bits long integer
2. Check with the Fermat Primality Test if the randomly chosen integer is prime
3. If not, repeat step 1 & 2

In [81]:
def generate_large_prime(nbits:int, s:int)->int:
    p_=random.randint(2**(nbits-1),2**(nbits));
    while not fermat_primality_test(p_,s):
        p_=random.randint(2**nbits,2**(nbits+1)-1);
    return p_

p=generate_large_prime(512,1000)
print("- p=",hex(p))
print("- len(p)=",len(bin(p))-2)
print("- is prime?",fermat_primality_test(p,1000));

q=generate_large_prime(512,1000)
print("- q=",hex(q))
print("- len(q)=",len(bin(q))-2)
print("- is prime?",fermat_primality_test(q,1000));

- p= 0x1c85b27a3312cc55ec69a081aca8d57d4e1ae2b8630dd5cd8acf39787274413ec65676ac1988dd6c58977818ba2b7f9c232ffe8290aa418ba5353ecb6968b6523
- len(p)= 513
- is prime? True
- q= 0x1e1eb34f2955198339356e9a4bd307e35d36108a99b2827a0c48bdb0c4ba7e726e494eb4a78217b39ffaa7c389bfa02eb7f535e62d973fab9434b9d04167e75ab
- len(q)= 513
- is prime? True


## Step 2: Computing n

For this step, we can simply multiply p and q using the built-in $*$ operator:

In [84]:
n = p*q
print("-n=",hex(n))

-n= 0x35b168f89f69a636d6c2c3c3174e69684af1156cf3709acbe5e6eea18cc24af8db45633795f67cd33b9eed9677537306421101129f34c1c2578c7f9b3eb8e58f132bac48861973a6ccb6417bd160a7614806628712cd617be336302e0064f0eed3fb9de186d79a4e288dba4cc291dadb1e192ee6f37616df8d881af9d0d8f8d61


## Step 3: Computing $\phi(n)$

Same remark for this step, we can simply use the built-in $*$ operator to compute $\phi(n)$:

In [87]:
phi=(p-1)*(q-1);
print("phi(n)=",hex(phi));

phi(n)= 0x35b168f89f69a636d6c2c3c3174e69684af1156cf3709acbe5e6eea18cc24af8db45633795f67cd33b9eed9677537306421101129f34c1c2578c7f9b3eb8e58ed887467f29b18dcda717325fd8e4ca009cb56f44160d09344c1e3904c936313d9f5bd880c5cca52e2ffb9a707ea6bb1042f3fa7e353495a8541e225e26085b294


## Step 4: Selecting the public key exponent e

This step is a bit tricky. We first need to randomly chose a number in the range $[1,\phi(n)-1]$ and then make sure that it is reversible, in other words that $e^-1$ exists in $\mathbb{Z}_{\phi_n}$. This ensures that the private key $d$ exists.
In order to make this verification, we will make use of the "Extended Euclidian Algorithm" that will also be used in the private key computation.

### Extended Euler Algorithm

![Extended Euler Algorithm](007.png "Extended Euler Algorithm")

In [115]:
def extended_euler_algorithm(r0:int, r1:int):
    s=[1,0];
    t=[0,1];
    r=[r0,r1];
    i=1;
    while r[i]!=0:
        i = i+1;
        r.append(r[i-2]%r[i-1]);
        q = (r[i-2]-r[i])//r[i-1];
        s.append(s[i-2]-q*s[i-1]);
        t.append(t[i-2]-q*t[i-1]);
    return (r[i-1], s[i-1], t[i-1])

1 -0xd1884939ce368b56c043f1d9a987942f008cb1d8fdc00f90ee172095af4abb1bdc061453231587d2911e03831052847c0592a4505ef16861243f324ffd0ae1796ca8578cdc70f94100884a3203c012b5811469afce15814b04030033b994083b2daff847ecdbc33ea284249cbcda3db3ccc8dd47b0d22faf808d8d4aa94cb27 0x416a9a8bb2ffa31223cd8ef0139f2c83a5d8e4b06033cb36a83068186973d299d1de8293dc83c76f4125307e90de61db929ff590896d46fc1ea11257611ece283d49c323d264cc7daba637fd0c8c4a3eb3c242026f16151216b51dab15991562847ebe2c6d8bda9b275054c8f827e6d77b0f83a6e6e9741efbaa7b5dae959e 0xc21a93ee751a8d4fbfd77285d79d6768c58ebf283743d2889a395f266c78f4a28e86f545960c2ce01eb8ad5246905163b28d0b8baabb959cc03f4ec499186168ae9ed6d88058898907e61c7cccc584d65d801cfe32dfc983707f87f5aa6ae4b9e77b9ce630e2c0df05841b5e4984d059a35d7270d500514891f7b77b804bed81


### Testing the Extended Euclidian Algorithm


Let's test the algorithm to assess that $gcd(e, \phi(n))=1$.

In [116]:
p=0xE0DFD2C2A288ACEBC705EFAB30E4447541A8C5A47A37185C5A9CB98389CE4DE19199AA3069B404FD98C801568CB9170EB712BF10B4955CE9C9DC8CE6855C6123;
q=0xEBE0FCF21866FD9A9F0D72F7994875A8D92E67AEE4B515136B2A778A8048B149828AEA30BD0BA34B977982A3D42168F594CA99F3981DDABFAB2369F229640115;
phi=(p-1)*(q-1);
e=0x40B028E1E4CCF07537643101FF72444A0BE1D7682F1EDB553E3AB4F6DD8293CA1945DB12D796AE9244D60565C2EB692A89B8881D58D278562ED60066DD8211E67315CF89857167206120405B08B54D10D4EC4ED4253C75FA74098FE3F7FB751FF5121353C554391E114C85B56A9725E9BD5685D6C9C7EED8EE442366353DC39;
gcd, _, _ = extended_euler_algorithm(e,phi);
print("gcd(e,phi(n))=",gcd);

gcd(e,phi(n))= 1


## Step 5: Computing the private key d

In order to compute the private key, we can use the parameter $t$ obtained from the EEA as follow: $d=t \ mod(\phi(n))$

In [118]:
p=0xE0DFD2C2A288ACEBC705EFAB30E4447541A8C5A47A37185C5A9CB98389CE4DE19199AA3069B404FD98C801568CB9170EB712BF10B4955CE9C9DC8CE6855C6123;
q=0xEBE0FCF21866FD9A9F0D72F7994875A8D92E67AEE4B515136B2A778A8048B149828AEA30BD0BA34B977982A3D42168F594CA99F3981DDABFAB2369F229640115;
phi=(p-1)*(q-1);
e=0x40B028E1E4CCF07537643101FF72444A0BE1D7682F1EDB553E3AB4F6DD8293CA1945DB12D796AE9244D60565C2EB692A89B8881D58D278562ED60066DD8211E67315CF89857167206120405B08B54D10D4EC4ED4253C75FA74098FE3F7FB751FF5121353C554391E114C85B56A9725E9BD5685D6C9C7EED8EE442366353DC39;
gcd, s, _ = extended_euler_algorithm(e,phi);
d=s%phi;
print("d=",hex(d));

d= 0xc21a93ee751a8d4fbfd77285d79d6768c58ebf283743d2889a395f266c78f4a28e86f545960c2ce01eb8ad5246905163b28d0b8baabb959cc03f4ec499186168ae9ed6d88058898907e61c7cccc584d65d801cfe32dfc983707f87f5aa6ae4b9e77b9ce630e2c0df05841b5e4984d059a35d7270d500514891f7b77b804bed81


# Full Protocol in Action

Now that we have gathered all the pieces from the puzzle, we can now put them together and see how the RSA key exchange would work.

# Digital Signature